# Model training notebook
This notebook is made for training the TDNN models from XVector.py module

In this notebook we carry out the grid search over the model's hyperparameters defined in grid dictionary and save the results as tensorboard runs

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Number of GPU:  1
GPU Name:  NVIDIA GeForce RTX 2060
Using device: cuda


In [3]:
import pandas as pd
import numpy as np
import mne
import os
import matplotlib.pyplot as plt

In [4]:
from eeg_lib.commons.constant import DATASETS_FOLDER
from eeg_lib.data.data_loader.EEGDataExtractor import EEGDataExtractor

In [5]:
from eeg_lib.utils.engine import create_user_profiles

In [6]:
from eeg_lib.utils.helpers import compute_genuine_imposter_distances, compute_threshold_metrics, compute_f1_vs_threshold, split_test_data_for_verification

In [7]:
from eeg_lib.utils.visualisations import plot_distance_distribution_on_ax, plot_threshold_metrics, plot_f1_vs_threshold, plot_distance_distribution_return, plot_f1_vs_threshold_return, plot_threshold_metrics_return

In [8]:
DATA_DIR = f"{DATASETS_FOLDER}/Kolory/"

extractor = EEGDataExtractor(data_dir=DATA_DIR, tmax=5.0, lfreq=7.0)
eeg_df, participants_info = extractor.extract_dataframe()

Opening raw data file C:\Users\JS\Desktop\Neuron\NeuroGuard\artificial-intelligence\eeg_lib/data/datasets/Kolory/011595b1@1651.fif...
    Range : 0 ... 207624 =      0.000 ...   830.496 secs
Ready.
Reading 0 ... 207624  =      0.000 ...   830.496 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 413 samples (1.652 s)

Filtering raw data in 1 contiguous segment


C:\Users\JS\Desktop\Neuron\NeuroGuard\artificial-intelligence\eeg_lib\data\data_loader\EEGDataExtractor.py:53: RuntimeWarning: This filename (C:\Users\JS\Desktop\Neuron\NeuroGuard\artificial-intelligence\eeg_lib/data/datasets/Kolory/011595b1@1651.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg = mne.io.read_raw_fif(file_path, preload=True)


Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 s)

Used Annotations descriptions: ['blue', 'break', 'gray', 'green', 'red', 'yellow']
Not setting metadata
232 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 232 events and 1251 original time points ...
0 bad epochs dropped
Opening raw data file C:\Users\JS\Desktop\Neuron\NeuroGuard\artificial-intelligence\eeg_lib/data/datasets/Kolory/022e8467@1910.fif...
    Range : 0 ... 212349 =      0.000 ...   849.396 sec

C:\Users\JS\Desktop\Neuron\NeuroGuard\artificial-intelligence\eeg_lib\data\data_loader\EEGDataExtractor.py:53: RuntimeWarning: This filename (C:\Users\JS\Desktop\Neuron\NeuroGuard\artificial-intelligence\eeg_lib/data/datasets/Kolory/022e8467@1910.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg = mne.io.read_raw_fif(file_path, preload=True)


Opening raw data file C:\Users\JS\Desktop\Neuron\NeuroGuard\artificial-intelligence\eeg_lib/data/datasets/Kolory/06f240e9@1215.fif...
    Range : 0 ... 184624 =      0.000 ...   738.496 secs
Ready.
Reading 0 ... 184624  =      0.000 ...   738.496 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 413 samples (1.652 s)

Filtering raw data in 1 contiguous segment


...

- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 413 samples (1.652 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 s)

Used Annotations descriptions: ['blue', 'break', 'gray', 'green', 

C:\Users\JS\Desktop\Neuron\NeuroGuard\artificial-intelligence\eeg_lib\data\data_loader\EEGDataExtractor.py:53: RuntimeWarning: This filename (C:\Users\JS\Desktop\Neuron\NeuroGuard\artificial-intelligence\eeg_lib/data/datasets/Kolory/fd8a3308@1135.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg = mne.io.read_raw_fif(file_path, preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 413 samples (1.652 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband

C:\Users\JS\Desktop\Neuron\NeuroGuard\artificial-intelligence\eeg_lib\data\data_loader\EEGDataExtractor.py:53: RuntimeWarning: This filename (C:\Users\JS\Desktop\Neuron\NeuroGuard\artificial-intelligence\eeg_lib/data/datasets/Kolory/ffae50df@1712.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg = mne.io.read_raw_fif(file_path, preload=True)


- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 413 samples (1.652 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 s)

Used Annotations descriptions: ['blue', 'break', 'gray', 'green', 'red', 'yellow']
Not setting metadata
232 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 232 events and 1251 original time points ...
0 bad epochs droppe

In [9]:
import torch.optim as optim
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from matplotlib.colors import ListedColormap

In [10]:
from sklearn.model_selection import ParameterGrid

In [11]:
import torch.nn as nn

In [12]:
from eeg_lib.data.data_loader.TDNNFeatures import extract_features, extract_psd_features
from eeg_lib.data.TDNNDataset import TDNNDataset, get_dataset
from eeg_lib.models.verification.XVector import XVectorEmbeddingModel
from eeg_lib.losses.ProxyNCALoss import ProxyNCALoss
from eeg_lib.utils.visualisations import plot_tsne
from eeg_lib.utils.visualisations import create_handles
from eeg_lib.utils.helpers import split_train_test
from eeg_lib.models.similarity.centroids import SimilarityCentroidsVerifier, get_accuracy

In [13]:
from torch.utils.tensorboard import SummaryWriter

In [14]:
from torch.optim.lr_scheduler import StepLR, ExponentialLR, ReduceLROnPlateau

In [15]:
from eeg_lib.models.verification.XVector import get_ecapa_model, get_standard_model, pretrain, fine_tune, create_embeddings, fine_tune_arcface

grid with defined available hyperparameters for the TDNN model (ECAPA_TDNNv2 in particular)

In [16]:
RUNS_FOLDER = "runs_ECAPA"
APPENDIX = "ECAPAv3"

In [31]:
grid = {
    "batch_size" : [64],
    "softmax_learning_rate" : [0.001],
    "proxy_learning_rate" : [0.001],
    "softmax_epochs" : [15],
    "proxy_epochs" : [60],
    "softmax_learning_rate_decay" : [0.95],
    "proxy_learning_rate_decay" : [0.95],
    "augmentation" : [True],
    "std" : [0.02],
    "embedding_dim" : [256],
    "dropout_rate" : [0.25],
    "scale" : [10],
    "margin" : [0.1],
    "layer1_filters" : [512],
    "layer2_filters" : [512],
    "layer3_filters" : [1024],
    "layer4_filters" : [1024],
    "layer5_filters" : [1500],
    "layer_1_dilatation" : [1],
    "layer_2_dilatation" : [2],
    "layer_3_dilatation" : [3],
    "layer_1_stride" : [1],
    "layer_2_stride" : [1],
    "layer_3_stride" : [2],
    "no_norm" : [True]
}

Data loading and preprocessing

In [18]:
X_train_tmp, X_test_tmp, y_train_tmp, y_test_tmp = split_train_test(eeg_df=eeg_df,test_size=0.2, random_state=42)

Training set participants: ['fd8a3308@1135' 'e08138e2@1731' '541c91f2@1456' '011595b1@1651'
 '2718372d@1400' '6e542bc2@0845' '2882ae26@1441' '548fd734@1628'
 '51ec2c20@0923' 'bf2d2193@1638' '022e8467@1910' '06f240e9@1215'
 'e283301e@1606' '25d0bdb3@1318' 'b34b1427@0906' 'e43a9f9f@0941'
 '8dca0725@1418' 'ffae50df@1712' '9e8bae0e@1828' '36eea4bb@1519'
 '46607ce4@1717' '54e60118@1339' '90441f44@1643' '3033b74a@1626']
Test set participants: ['f82b5699@1757' '6d9a8b86@1613' 'd87e1bd3@1806' '8bd3032e@1746'
 '39285860@1825' '446b3735@1618']
Training labels: ['011595b1@1651' '022e8467@1910' '06f240e9@1215' '25d0bdb3@1318'
 '2718372d@1400' '2882ae26@1441' '3033b74a@1626' '36eea4bb@1519'
 '46607ce4@1717' '51ec2c20@0923' '541c91f2@1456' '548fd734@1628'
 '54e60118@1339' '6e542bc2@0845' '8dca0725@1418' '90441f44@1643'
 '9e8bae0e@1828' 'b34b1427@0906' 'bf2d2193@1638' 'e08138e2@1731'
 'e283301e@1606' 'e43a9f9f@0941' 'fd8a3308@1135' 'ffae50df@1712']
Test labels: ['39285860@1825' '446b3735@1618' '6d9a8

In [19]:
epoch_length = X_train_tmp[0].shape[1]
div, mod = divmod(epoch_length, 50)
truncated_length = div*50

In [20]:
extracted_X_train = []
for epoch in X_train_tmp:
    extracted_X_train.append(extract_features(epoch, fs=250, trunc=truncated_length).T) 

In [21]:
extracted_X_test = []
for epoch in X_test_tmp:
    extracted_X_test.append(extract_features(epoch, fs=250, trunc=truncated_length).T)

In [22]:
le_train = LabelEncoder()
le_train.fit(y_train_tmp)
y_train_encoded = le_train.transform(y_train_tmp)

le_test = LabelEncoder()
le_test.fit(y_test_tmp)
y_test_encoded = le_test.transform(y_test_tmp)

In [23]:
X_train = np.array(extracted_X_train)
X_test = np.array(extracted_X_test)

In [24]:
scalers = {}
X_train_norm = np.empty_like(X_train)
X_test_norm = np.empty_like(X_test)

In [25]:
# scaling per feature
for f in range(X_train.shape[1]):
    scalers[f] = StandardScaler().fit(X_train[:, f, :])
    X_train_norm[:, f,:] = scalers[f].transform(X_train[:, f, :])
    X_test_norm[:, f, :] = scalers[f].transform(X_test[:, f, :])

In [26]:
X_train_norm.shape

(5562, 28, 25)

In [27]:
num_train_classes = len(np.unique(y_train_encoded))

In [28]:
custom_colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4',
                 '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff',
                 '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1',
                 '#000075', '#808080', '#ffffff', '#000000', '#a9a9a9', '#ff69b4',
                 '#b0e0e6', '#32cd32', '#ff4500', '#da70d6', '#ff1493', '#7fffd4']
cmap = ListedColormap(custom_colors[:30])

custom_colors = ['#e6194b',  # Red
                 '#3cb44b',  # Green
                 '#4363d8',  # Blue
                 '#ffe119',  # Yellow
                 '#911eb4',  # Purple
                 '#f58231']  # Orange

cmap2 = ListedColormap(custom_colors[:6])

In [29]:
train_handles = create_handles(y_train_tmp, cmap)
test_handles = create_handles(y_test_tmp, cmap2)

In the tensorboard run we save:
- pretaraining loss
- pretraining accuracy
- finetuning loss
- centroid based accuracy metric for train and test data
- test and train TSNE figures
- genuine vs imposter plots for both cosine and euclidean distances train and test
- threshold values, f1 score at threshold, accuracy at threshold, for cosine, euclidean metrics, train and test data
- plots for f1 vs threshold
- plots for accuracy vs threshold

In [32]:
for run_idx, hparams in enumerate(ParameterGrid(grid), start=1):
    print("RUN NUMBER: {}".format(run_idx))
    run_name = f"run_{run_idx}"
    writer = SummaryWriter(log_dir=f"{RUNS_FOLDER}/{run_name}_{APPENDIX}")
    # print(hparams)
    writer.add_hparams(hparams, {})
    
    train_loader = get_dataset(hparams,X_train_norm, y_train_encoded)
    model = pretrain(hparams, device, X_train_norm.shape[1], num_train_classes, train_loader, writer, "ECAPA2").to(device)
    model = fine_tune_arcface(model, hparams, device, train_loader, num_train_classes, writer).to(device)
    
    # checkpoint = {
    #     "model_state_dict": model.state_dict()
    # }
    # torch.save(checkpoint, "saved_models/ECAPA2/init_model_for_featuregrid.pth")
    model.to('cpu')
    embd_train, embd_test = create_embeddings(model, X_train_norm, X_test_norm,hparams)
    
    centroid_train_acc, centroid_test_acc = get_accuracy(embd_train, embd_test, y_train_encoded, y_test_encoded)
    
    writer.add_scalar("Final_Centroid_Accuracy/train", centroid_train_acc)
    writer.add_scalar("Final_Centroid_Accuracy/test", centroid_test_acc)
    
    tsne_train = TSNE(n_components=2, random_state=42).fit_transform(embd_train)
    tsne_test = TSNE(n_components=2, random_state=42).fit_transform(embd_test)
    
    scaler = MinMaxScaler(feature_range=(0,1))
    tsne_train_scaled = scaler.fit_transform(tsne_train)
    tsne_test_scaled = scaler.transform(tsne_test)
    
    fig_train = plot_tsne(tsne_train_scaled,
          cmap,
          y_train_encoded,
          handles=train_handles,
          alpha=0.5,
          title="TSNE Visualization training data XVector",
          xlabel="TSNE embedding dimension 1",
          ylabel="TSNE embedding dimension 2",
          centroids=None,
          return_fig=True)
    # plt.show(fig_train)
    writer.add_figure("tsne_train", fig_train)
    fig_test = plot_tsne(tsne_test_scaled,
          cmap2,
          y_test_encoded,
          handles=test_handles,
          alpha=0.5,
          title="TSNE Visualization test data XVector",
          xlabel="TSNE embedding dimension 1",
          ylabel="TSNE embedding dimension 2",
          centroids=None, return_fig=True )
    # plt.show(fig_test)
    embd_unit = embd_train / np.linalg.norm(embd_train, axis=1, keepdims=True)
    writer.add_figure("tsne_test", fig_test)
    
    fig = plot_distance_distribution_return(
        embeddings=embd_unit,
        participant_ids=np.array(y_train_encoded),
        distance_type="euclidean",
        bins=30
    )
    # plt.show(fig)
    writer.add_figure("distance_distribution_euclidean", fig)
    
    fig = plot_distance_distribution_return(
        embeddings=embd_train,
        participant_ids=np.array(y_train_encoded),
        distance_type="cosine",
        bins=30
    )
    # plt.show(fig)
    writer.add_figure("distance_distribution_cosine", fig)
    
    # train
    user_profiles = create_user_profiles(embd_train, np.array(y_train_encoded))
    for metric in ("euclidean", "cosine"):
        print(metric)
        print("train")
        genuine_dists, imposter_dists = compute_genuine_imposter_distances(
                embeddings=embd_train ,
                ids=np.array(y_train_encoded),
                user_profiles=user_profiles,
                distance_metric=metric
            )
        
        (
            thresholds, fnr_list, fpr_list, acc_list,
            best_T, best_fnr, best_fpr, best_acc
        ) = compute_threshold_metrics(genuine_dists, imposter_dists, num_thresholds=200)
        
        writer.add_scalar(f"{metric}_best_threshold_train", best_T)
        writer.add_scalar(f"{metric}_best_acc_train", best_acc)
        writer.add_scalar(f"{metric}_FNR/FPR_threshold_train", best_fnr)
        
        
        fig = plot_threshold_metrics_return(
            thresholds, fnr_list, fpr_list, acc_list,
            best_T, best_fnr, best_fpr, best_acc
        )
        # plt.show(fig)
        writer.add_figure(f"threshold_metrics_train_{metric}", fig)
        
        thresholds, f1_list, best_T, best_f1 = compute_f1_vs_threshold(
            genuine_dists, imposter_dists, num_thresholds=300
        )
        writer.add_scalar(f"{metric}_best_f1_train", best_f1)
        
        fig = plot_f1_vs_threshold_return(thresholds, f1_list, best_T, best_f1)
        writer.add_figure(f"f1_vs_threshold_train_{metric}", fig)
        # plt.show(fig)
    

    # test
    profile_embd, profile_ids, verify_embd, verify_ids = split_test_data_for_verification(
        embd_test, np.array(y_test_encoded), profile_ratio=0.6
    )
    
    test_user_profiles = create_user_profiles(profile_embd, profile_ids)
    for metric in ("euclidean", "cosine"):
    
        genuine_dists, imposter_dists = compute_genuine_imposter_distances(
            embeddings=verify_embd,
            ids=verify_ids,
            user_profiles=test_user_profiles,
            distance_metric=metric
        )
        
        (
            test_thresholds, test_fnr_list, test_fpr_list, test_acc_list,
            test_best_T, test_best_fnr, test_best_fpr, test_best_acc
        ) = compute_threshold_metrics(genuine_dists, imposter_dists, num_thresholds=200)
        writer.add_scalar(f"{metric}_best_threshold_test", test_best_T)
        writer.add_scalar(f"{metric}_best_acc_test", test_best_acc)
        writer.add_scalar(f"{metric}_FNR/FPR_th0reshold_test", test_best_fnr)
        
        fig = plot_threshold_metrics_return(
            test_thresholds, test_fnr_list, test_fpr_list, test_acc_list,
            test_best_T, test_best_fnr, test_best_fpr, test_best_acc
        )
        # plt.show(fig)
        writer.add_figure(f"threshold_metrics_test_{metric}", fig)
        
        test_thresholds_f1, test_f1_list, test_best_T_f1, test_best_f1 = compute_f1_vs_threshold(
            genuine_dists, imposter_dists, num_thresholds=300
        )
        writer.add_scalar(f"{metric}_best_f1_test", test_best_f1)
        
        
        fig = plot_f1_vs_threshold_return(test_thresholds_f1, test_f1_list, test_best_T_f1, test_best_f1)
        # plt.show(fig)
        writer.add_figure(f"f1_vs_threshold_test_{metric}", fig)
    writer.close()                                                                                    

RUN NUMBER: 1
[Pretrain] Epoch 1/15  Loss=2.2754  Acc=0.2469
[Pretrain] Epoch 2/15  Loss=1.2839  Acc=0.5613
[Pretrain] Epoch 3/15  Loss=0.8417  Acc=0.7138
[Pretrain] Epoch 4/15  Loss=0.6329  Acc=0.7862
[Pretrain] Epoch 5/15  Loss=0.4790  Acc=0.8344
[Pretrain] Epoch 6/15  Loss=0.3898  Acc=0.8626
[Pretrain] Epoch 7/15  Loss=0.3363  Acc=0.8833
[Pretrain] Epoch 8/15  Loss=0.3033  Acc=0.8979
[Pretrain] Epoch 9/15  Loss=0.2439  Acc=0.9155
[Pretrain] Epoch 10/15  Loss=0.2248  Acc=0.9257
[Pretrain] Epoch 11/15  Loss=0.2055  Acc=0.9256
[Pretrain] Epoch 12/15  Loss=0.1786  Acc=0.9371
[Pretrain] Epoch 13/15  Loss=0.1452  Acc=0.9513
[Pretrain] Epoch 14/15  Loss=0.1283  Acc=0.9534
[Pretrain] Epoch 15/15  Loss=0.1235  Acc=0.9567
[ArcFace Fine‑tune] Epoch 1/60  Loss=0.9914
[ArcFace Fine‑tune] Epoch 2/60  Loss=0.3697
[ArcFace Fine‑tune] Epoch 3/60  Loss=0.2786
[ArcFace Fine‑tune] Epoch 4/60  Loss=0.2206
[ArcFace Fine‑tune] Epoch 5/60  Loss=0.1701
[ArcFace Fine‑tune] Epoch 6/60  Loss=0.1935
[ArcFace Fi